In [5]:
import simpy
import random
import numpy as np
import heapq

In [6]:
# Define system parameters
ARRIVAL_RATE = 10  # vehicles per minute (lambda)
SERVICE_RATE = 15  # vehicles per minute (mu)
NUM_LANES = 2  # Number of servers (traffic lanes)
SIM_TIME = 120  # Simulation time in minutes
GREEN_LIGHT_DURATION = 30  # Green light duration in seconds
RED_LIGHT_DURATION = 30  # Red light duration in seconds

In [7]:
class TrafficIntersection:
    def __init__(self, env, num_lanes, service_rate):
        self.env = env
        self.lanes = simpy.Resource(env, num_lanes)
        self.service_rate = service_rate
        self.queue_length = []  # Explicit queue length tracking
        self.queue = []  # Explicit queue storage
    
    def serve_vehicle(self):
        service_time = random.expovariate(self.service_rate)
        yield self.env.timeout(service_time)

In [8]:
# Traffic signal controller
def traffic_light_controller(env, intersection):
    while True:
        yield env.timeout(GREEN_LIGHT_DURATION)  # Green light period
        print(f"{env.now}: GREEN light - Vehicles moving")
        yield env.timeout(RED_LIGHT_DURATION)  # Red light period
        print(f"{env.now}: RED light - Vehicles stopping")


In [9]:
# Vehicle arrival process
def vehicle_generator(env, intersection, arrival_rate):
    while True:
        yield env.timeout(random.expovariate(arrival_rate))
        env.process(vehicle_process(env, intersection))


In [10]:
# Vehicle queuing process
def vehicle_process(env, intersection):
    arrival_time = env.now
    intersection.queue.append(arrival_time)  # Track vehicle entering queue
    with intersection.lanes.request() as request:
        yield request
        wait_time = env.now - arrival_time
        intersection.queue_length.append(len(intersection.queue))  # Track queue length dynamically
        intersection.queue.pop(0)  # Remove from queue once served
        WAIT_TIMES.append(wait_time)
        yield env.process(intersection.serve_vehicle())

In [11]:
# Event Scheduler for defined events
class EventScheduler:
    def __init__(self, env):
        self.env = env
        self.event_queue = []  #Using a priority queue for events

    # Scheduling the event
    def schedule_event(self, time, event_type, details=None):
        heapq.heappush(self.event_queue, (time, event_type, details))

    def process_events(self):
        while self.event_queue:
            event_time, event_type, details = heapq.heappop(self.event_queue)
            yield self.env.timeout(event_time - self.env.now)
            print(f"Time {self.env.now}: Event '{event_type}' occurred. Details: {details}")

In [12]:
# Initialize simulation environment
env = simpy.Environment()
scheduler = EventScheduler(env)
intersection = TrafficIntersection(env, NUM_LANES, SERVICE_RATE)
WAIT_TIMES = []

In [13]:
# Sample Events to Schedule
scheduler.schedule_event(3, "Traffic Light Change", "Green → Yellow")
scheduler.schedule_event(5, "Car Arrival", "Lane A")
scheduler.schedule_event(7, "Traffic Light Change", "Yellow → Red")
scheduler.schedule_event(10, "Car Departure", "Lane A")

In [14]:
env.process(vehicle_generator(env, intersection, ARRIVAL_RATE))
env.process(traffic_light_controller(env, intersection))
env.process(scheduler.process_events())
env.run(until=SIM_TIME)


Time 3: Event 'Traffic Light Change' occurred. Details: Green → Yellow
Time 5: Event 'Car Arrival' occurred. Details: Lane A
Time 7: Event 'Traffic Light Change' occurred. Details: Yellow → Red
Time 10: Event 'Car Departure' occurred. Details: Lane A
30: GREEN light - Vehicles moving
60: RED light - Vehicles stopping
90: GREEN light - Vehicles moving


In [15]:
# Performance Metrics
avg_wait_time = np.mean(WAIT_TIMES)
queue_length = len(WAIT_TIMES) / SIM_TIME  # Approximate queue length
system_utilization = (ARRIVAL_RATE / (NUM_LANES * SERVICE_RATE))

In [16]:
print(f"Average Wait Time: {avg_wait_time:.2f} minutes")
print(f"Approximate Queue Length: {queue_length:.2f} vehicles")
print(f"System Utilization: {system_utilization:.2f}")

Average Wait Time: 0.01 minutes
Approximate Queue Length: 9.82 vehicles
System Utilization: 0.33
